# Recommender System Early Demo Code #

### import necessary libraries ###

In [1]:
import pymysql
import pandas as pd

# jaccard similarity 
from scipy.spatial.distance import pdist, squareform

# 引入套件計算以餘弦相似度
from sklearn.metrics.pairwise import cosine_similarity

### Making a sql connection to database ###

In [2]:
conn = pymysql.connect(host = 'cgi101-db.ckelxbpeajyc.ap-northeast-3.rds.amazonaws.com',
                                    database='project_v5',
                                    user='',
                                    password='')

### Create a cursor ###

In [3]:
cursor = conn.cursor()

### Execute SQL ###

cursor.execute("""SELECT
	concat(dname,tpname) drink_name, dname,typename, dprice,tpname AS topping,  tpprice,  (dprice+tpprice) AS total 
FROM 
	(SELECT dname,dprice,typename FROM project_v5.dmenu 
			where dprice > 1  
			AND dname NOT LIKE '%元' and dname NOT LIKE '%特價%' and typename NOT LIKE '%外送%') d
		CROSS JOIN
	(SELECT tpname, tpprice FROM project_v5.toppings where tpprice > 1
			AND tpno < '000037' and tpno not in ('000024','000026','000027'))t  order by drink_name;""")

In [4]:
cursor.execute("""SELECT distinct(tt.d_name) dname,dprice, tt.typename
FROM (SELECT if(INSTR(t.dname,'(XL)'),SUBSTR(t.dname,1,INSTR(t.dname,'(XL)')-1),t.dname) d_name, dprice,
	if(INSTR(t.typename,'(XL)'),SUBSTR(t.typename,1,INSTR(t.typename,'(XL)')-1),t.typename) typename
FROM (SELECT if(INSTR(dname,'(L)'),SUBSTR(dname,1,INSTR(dname,'(L)')-1),dname) dname, dprice,
if(INSTR(typename,'(L)'),SUBSTR(typename,1,INSTR(typename,'(L)')-1),typename) typename FROM project_v5.dmenu ) t
) tt where tt.d_name NOT LIKE '%元%' AND tt.d_name NOT IN('             ※','贈品','購物袋') AND TYPENAME != '外送平台' ORDER BY dname;
""")

150

### Creating DataFrame

In [5]:
# get column names
fields = [field_md[0] for field_md in cursor.description]
# zip column names and rows
result = [dict(zip(fields,row)) for row in cursor.fetchall()]
df = pd.DataFrame(result)

In [6]:
df =  df.iloc[0::2,:].reset_index(drop=True)

In [7]:
df

,dname,dprice,typename
0,仙草凍奶茶,40,奶茶
1,仙草甘茶,25,精選茗茶
2,仙草蜜,35,特調
3,仙草鮮奶凍,50,鮮奶
4,冬瓜仙草茶,30,調茶
...,...,...,...
70,鮮奶烏龍,45,鮮奶
71,鮮奶紅茶,45,鮮奶
72,鮮奶綠茶,45,鮮奶
73,鮮奶青茶,45,鮮奶


### Load the ingredient data

In [8]:
ingredient = pd.read_csv("data_from_shop.csv")
ingredient

,dname,無糖甜度(0~10),總甜度,熱品/冷品,主要成分,次要成分1,次要成分2,次要成分3,水果/成分,溫du
0,仙草凍奶茶,2,3,熱/冷,奶茶,NaN,仙草凍,NaN,NaN,偏冷
1,仙草甘茶,0,1,熱/冷,NaN,仙草茶,NaN,NaN,NaN,偏冷
2,仙草蜜,10,3,冷,蜜茶,NaN,仙草凍,NaN,NaN,偏冷
3,仙草鮮奶凍,5,2,熱/冷,NaN,鮮奶,仙草凍,NaN,NaN,偏冷
4,冬瓜仙草茶,3,2,熱/冷,冬瓜茶,仙草茶,NaN,NaN,NaN,偏冷
...,...,...,...,...,...,...,...,...,...,...
70,鮮奶烏龍,0,2,熱/冷,烏龍茶,鮮奶,NaN,NaN,NaN,偏冷
71,鮮奶紅茶,0,2,熱/冷,紅茶,鮮奶,NaN,NaN,NaN,偏冷
72,鮮奶綠茶,0,2,熱/冷,綠茶,鮮奶,NaN,NaN,NaN,偏冷
73,鮮奶青茶,0,2,熱/冷,青茶,鮮奶,NaN,NaN,NaN,偏冷


### Merge two dfs (inner join)

In [9]:
df_final = pd.merge(df,ingredient, how='inner', on='dname')
df_final

,dname,dprice,typename,無糖甜度(0~10),總甜度,熱品/冷品,主要成分,次要成分1,次要成分2,次要成分3,水果/成分,溫du
0,仙草凍奶茶,40,奶茶,2,3,熱/冷,奶茶,NaN,仙草凍,NaN,NaN,偏冷
1,仙草甘茶,25,精選茗茶,0,1,熱/冷,NaN,仙草茶,NaN,NaN,NaN,偏冷
2,仙草蜜,35,特調,10,3,冷,蜜茶,NaN,仙草凍,NaN,NaN,偏冷
3,仙草鮮奶凍,50,鮮奶,5,2,熱/冷,NaN,鮮奶,仙草凍,NaN,NaN,偏冷
4,冬瓜仙草茶,30,調茶,3,2,熱/冷,冬瓜茶,仙草茶,NaN,NaN,NaN,偏冷
...,...,...,...,...,...,...,...,...,...,...,...,...
70,鮮奶烏龍,45,鮮奶,0,2,熱/冷,烏龍茶,鮮奶,NaN,NaN,NaN,偏冷
71,鮮奶紅茶,45,鮮奶,0,2,熱/冷,紅茶,鮮奶,NaN,NaN,NaN,偏冷
72,鮮奶綠茶,45,鮮奶,0,2,熱/冷,綠茶,鮮奶,NaN,NaN,NaN,偏冷
73,鮮奶青茶,45,鮮奶,0,2,熱/冷,青茶,鮮奶,NaN,NaN,NaN,偏冷


### Checking Null Values

In [10]:
df_final.columns[1:]


Index(['dprice', 'typename', '無糖甜度(0~10)', '總甜度', '熱品/冷品', '主要成分', '次要成分1',
       '次要成分2', '次要成分3', '水果/成分', '溫du'],
      dtype='object')

### One-hot Encoding Using pd.get_dummies

In [11]:
encoded_df = pd.get_dummies(df_final,columns=df_final.columns[1:])

### Set drink_name as Index

In [12]:
encoded_df.set_index("dname",inplace=True)
encoded_df

,dprice_25,dprice_30,dprice_35,dprice_40,dprice_45,dprice_50,typename_咖啡,typename_多多,typename_奶茶,typename_巧克力,...,次要成分2_金桔,次要成分3_梅子,次要成分3_百香顆粒,次要成分3_薄荷糖漿,次要成分3_蜂蜜,次要成分3_養樂多,水果/成分_可可粉,水果/成分_水果類,溫du_偏冷,溫du_偏熱
dname,,,,,,,,,,,,,,,,,,,,,
仙草凍奶茶,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
仙草甘茶,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
仙草蜜,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
仙草鮮奶凍,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
冬瓜仙草茶,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
鮮奶烏龍,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
鮮奶紅茶,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
鮮奶綠茶,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


### Cosine Similarity

In [13]:
# 計算餘弦相似度，並會以 np.array 形式輸出儲存
cosine_similarity_array = cosine_similarity(encoded_df)

# 將資料轉換為 DF 型態以方便進行資料檢視與處理
cosine_similarity_df = pd.DataFrame(cosine_similarity_array,
                                    index=encoded_df.index,
                                    columns=encoded_df.index)

In [14]:
cosine_similarity_df

dname,仙草凍奶茶,仙草甘茶,仙草蜜,仙草鮮奶凍,冬瓜仙草茶,冬瓜檸檬,冬瓜茶,冬瓜調茶,咖啡凍奶茶,奶香青茶,...,香草咖啡,香草奶茶,香草巧克力,高山青茶,鮮奶冬瓜,鮮奶烏龍,鮮奶紅茶,鮮奶綠茶,鮮奶青茶,鮮葡萄柚綠
dname,,,,,,,,,,,,,,,,,,,,,
仙草凍奶茶,1.000000,0.267261,0.375000,0.375000,0.250000,0.117851,0.267261,0.250000,0.875000,0.375000,...,0.125000,0.625000,0.125000,0.267261,0.250000,0.250000,0.250000,0.250000,0.250000,0.235702
仙草甘茶,0.267261,1.000000,0.133631,0.267261,0.400892,0.125988,0.571429,0.267261,0.267261,0.400892,...,0.133631,0.133631,0.267261,0.857143,0.267261,0.400892,0.400892,0.400892,0.400892,0.377964
仙草蜜,0.375000,0.133631,1.000000,0.250000,0.125000,0.471405,0.267261,0.125000,0.250000,0.250000,...,0.000000,0.125000,0.000000,0.133631,0.125000,0.125000,0.125000,0.125000,0.125000,0.353553
仙草鮮奶凍,0.375000,0.267261,0.250000,1.000000,0.375000,0.235702,0.400892,0.375000,0.250000,0.375000,...,0.500000,0.250000,0.375000,0.267261,0.625000,0.625000,0.625000,0.625000,0.625000,0.117851
冬瓜仙草茶,0.250000,0.400892,0.125000,0.375000,1.000000,0.471405,0.534522,0.875000,0.250000,0.375000,...,0.250000,0.125000,0.125000,0.267261,0.500000,0.375000,0.375000,0.375000,0.375000,0.117851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
鮮奶烏龍,0.250000,0.400892,0.125000,0.625000,0.375000,0.235702,0.400892,0.375000,0.250000,0.500000,...,0.250000,0.125000,0.125000,0.400892,0.750000,1.000000,0.875000,0.875000,0.875000,0.235702
鮮奶紅茶,0.250000,0.400892,0.125000,0.625000,0.375000,0.235702,0.400892,0.375000,0.250000,0.500000,...,0.250000,0.125000,0.125000,0.400892,0.750000,0.875000,1.000000,0.875000,0.875000,0.235702
鮮奶綠茶,0.250000,0.400892,0.125000,0.625000,0.375000,0.235702,0.400892,0.375000,0.250000,0.500000,...,0.250000,0.125000,0.125000,0.400892,0.750000,0.875000,0.875000,1.000000,0.875000,0.353553


### '仙草凍奶茶'相似的前20個飲料

In [15]:
cosine_similarity_df.loc['仙草凍奶茶'].sort_values(ascending=False).head(20)

dname
仙草凍奶茶    1.000000
咖啡凍奶茶    0.875000
布丁奶茶     0.875000
椰果奶茶     0.750000
珍珠奶茶     0.750000
香草奶茶     0.625000
榛果奶茶     0.625000
芋香奶茶     0.625000
焦糖奶茶     0.625000
草莓奶茶     0.589256
蜂蜜奶綠     0.589256
阿薩姆奶茶    0.500000
碳焙烏龍奶    0.500000
薄荷奶綠     0.471405
奶香青茶     0.375000
巧克力奶茶    0.375000
仙草蜜      0.375000
仙草鮮奶凍    0.375000
薄荷巧克力    0.375000
茉香奶綠     0.375000
Name: 仙草凍奶茶, dtype: float64

In [16]:
def top_k_items(similarity_df,item_name,k):
    return similarity_df.loc[item_name].sort_values(ascending=False).head(k+1).index[1:].to_list()

In [17]:
print(top_k_items(cosine_similarity_df,'仙草凍奶茶',5))

['咖啡凍奶茶', '布丁奶茶', '椰果奶茶', '珍珠奶茶', '香草奶茶']


In [18]:
print(top_k_items(cosine_similarity_df,'鮮葡萄柚綠',5))

['檸檬鮮果汁', '檸檬蘆薈', '話梅檸檬', '荔枝蘆薈', '海尼根綠茶']


In [19]:
print(top_k_items(cosine_similarity_df,'椰果奶茶',5))

['珍珠奶茶', '仙草凍奶茶', '咖啡凍奶茶', '布丁奶茶', '芋香奶茶']


In [21]:
cosine_similarity_df.to_csv("drinks_similarity.csv")

In [28]:
df = pd.read_csv("drinks_similarity.csv",index_col=0)

In [29]:
df

,仙草凍奶茶,仙草甘茶,仙草蜜,仙草鮮奶凍,冬瓜仙草茶,冬瓜檸檬,冬瓜茶,冬瓜調茶,咖啡凍奶茶,奶香青茶,...,香草咖啡,香草奶茶,香草巧克力,高山青茶,鮮奶冬瓜,鮮奶烏龍,鮮奶紅茶,鮮奶綠茶,鮮奶青茶,鮮葡萄柚綠
dname,,,,,,,,,,,,,,,,,,,,,
仙草凍奶茶,1.000000,0.267261,0.375000,0.375000,0.250000,0.117851,0.267261,0.250000,0.875000,0.375000,...,0.125000,0.625000,0.125000,0.267261,0.250000,0.250000,0.250000,0.250000,0.250000,0.235702
仙草甘茶,0.267261,1.000000,0.133631,0.267261,0.400892,0.125988,0.571429,0.267261,0.267261,0.400892,...,0.133631,0.133631,0.267261,0.857143,0.267261,0.400892,0.400892,0.400892,0.400892,0.377964
仙草蜜,0.375000,0.133631,1.000000,0.250000,0.125000,0.471405,0.267261,0.125000,0.250000,0.250000,...,0.000000,0.125000,0.000000,0.133631,0.125000,0.125000,0.125000,0.125000,0.125000,0.353553
仙草鮮奶凍,0.375000,0.267261,0.250000,1.000000,0.375000,0.235702,0.400892,0.375000,0.250000,0.375000,...,0.500000,0.250000,0.375000,0.267261,0.625000,0.625000,0.625000,0.625000,0.625000,0.117851
冬瓜仙草茶,0.250000,0.400892,0.125000,0.375000,1.000000,0.471405,0.534522,0.875000,0.250000,0.375000,...,0.250000,0.125000,0.125000,0.267261,0.500000,0.375000,0.375000,0.375000,0.375000,0.117851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
鮮奶烏龍,0.250000,0.400892,0.125000,0.625000,0.375000,0.235702,0.400892,0.375000,0.250000,0.500000,...,0.250000,0.125000,0.125000,0.400892,0.750000,1.000000,0.875000,0.875000,0.875000,0.235702
鮮奶紅茶,0.250000,0.400892,0.125000,0.625000,0.375000,0.235702,0.400892,0.375000,0.250000,0.500000,...,0.250000,0.125000,0.125000,0.400892,0.750000,0.875000,1.000000,0.875000,0.875000,0.235702
鮮奶綠茶,0.250000,0.400892,0.125000,0.625000,0.375000,0.235702,0.400892,0.375000,0.250000,0.500000,...,0.250000,0.125000,0.125000,0.400892,0.750000,0.875000,0.875000,1.000000,0.875000,0.353553
